#DATAFRAME BverfG

In [ ]:
### GERMANY ###

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url_prefix = "https://www.bundesverfassungsgericht.de/SiteGlobals/Forms/Suche/Entscheidungensuche_Formular.html?gts=5403124_list%253DdateOfIssue_dt%252Basc&submit=Senden&dateAfter=01.01.2018&facettedEntscheidungstyp=Urteil&templateQueryString=EU&dateBefore=01.01.2023&language_=de"
decision_urls = []

for page_num in range(1,2):
    response = requests.get(url_prefix.format(page_num))
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    decisions_bverg = soup.find_all('span', class_="aktenzeichen")
    for x in decisions_bverg:
        parent_anchor = x.parent
        if parent_anchor and parent_anchor.has_attr('href'):
            decision_url = parent_anchor["href"]
            decision_urls.append(decision_url)

df = pd.DataFrame(decision_urls)

pd.set_option('display.max_colwidth', None)

decision_url_strings = []

for url in decision_urls:
    url_string = "https://www.bundesverfassungsgericht.de/" + url
    decision_url_strings.append(url_string)


df1 = pd.DataFrame(decision_url_strings, columns=['Decision URLs'])
print(df1)

In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

headings = []

for x in decision_url_strings:
    response = requests.get(x)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        indicator_element = soup.find('p', class_='cite')

        if indicator_element:
            element_text = indicator_element.get_text(strip=True)

            terms = element_text.split(' - ')
            headings.extend(terms)
        else:
            print("The element was not found on the webpage.")
    else:
        print(f"Failed to fetch the decision URL: {x}")


print(headings)
len(headings)

df = pd.DataFrame(columns=["Date", "Decision No.", "URL", "Chamber"])

for x in headings:
    date_pattern = r"vom (\d{1,2}\. \w+ \d{4})"
    decision_pattern = r"(\d+ [A-Za-z]+ \d+/\d+)"
    url_pattern = r"https:\/\/www\.bverfg\.de\/e\/[a-z0-9_]+\.[a-z]+"
    chamber_pattern = r"(Ersten|Zweiten) Senats"

    match = re.search(date_pattern, x)
    date = match.group(1) if match else None

    match = re.search(decision_pattern, x)
    decision_no = match.group(1) if match else None

    match = re.search(url_pattern, x)
    url = match.group() if match else None

    match = re.search(chamber_pattern, x)
    chamber = match.group(1) if match else None

    df = df.append({"Date": date, "Decision No.": decision_no, "URL": url, "Chamber": chamber}, ignore_index=True)

print(df)


import re
import pandas as pd
import requests
from bs4 import BeautifulSoup


def find_eu_law(text):
    eu_law_patterns = [
        r"Art\. 267 Abs\. 3 AEUV",
        r"Art\. \d+ Buchstabe [a-z]+ Verordnung \(EU\) Nr\. \d+/\d+ des Rates vom \d+\. [A-Za-z]+ \d+",
        r"Artikel \d+ des Vertrages über die Arbeitsweise der Europäischen Union",
        r"Art\. \d+ Abs\. \d+ [A-Za-z]+ \(EG\) Nr\. \d+/\d+ der Kommission",
        r"Art\. \d+ Abs\. \d+ [A-Za-z]+ EUV",
        r"Art\. \d+ Abs\. \d+ und Abs\. \d+ EUV",
        r"Verordnung <EG> Nr\. \d+/\d+",
        r"Art\. \d+ Abs\. \d+ AEUV",
        r"Beschluss \[EU\] \d+/\d+",
        r"<EU> \d+/\d+",
        r"Verordnung \(EU\) Nr\. \d+/\d+",
        r"ABl EU Nr\. L \d+ vom \d+\. [A-Za-z]+ \d+, S\. \d+",
        r"ABl EU Nr\. L \d+ vom \d+\. [A-Za-z]+ \d+",
        r"EUCO \d+/\d+",
        r"§ \d+ Abs\. \d+ [A-Za-z]+",
        r"ABl EU, L \d+",
        r"EU:C:\d+:\d+"
    ]
    eu_law_matches = []
    for pattern in eu_law_patterns:
        matches = re.findall(pattern, text)
        eu_law_matches.extend(matches)
    return eu_law_matches

for index, row in df.iterrows():
    url = row['URL']
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text()
        eu_law = list(set(find_eu_law(text)))
        eu_law_str = ', '.join(eu_law)
        df.at[index, 'EU law'] = eu_law_str
    else:
        print(f"Failed to fetch the URL: {url}")

print(df)


df['Top 50 Words'] = ''

import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')


excluded_words = ['abgabe', 'kommt', 'verfasssungschutzes', "genauigkeit", "antragsgegner","gilt", "hinreichend", "grundlage", "schutz", "januar", "gesetzgeber","gerichtshof", "programm", "maßnahme", "statistische", "parlament" ]
excluded_months = ['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 'August', 'September', 'Oktober', 'November', 'Dezember']

import itertools
combinations_of_3_letters = [''.join(comb) for comb in itertools.combinations('abcdefghijklmnopqrstuvwxyz', 3)]
excluded_words.extend(combinations_of_3_letters)

for i, url in enumerate(df['URL']):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text_content = " ".join([p.get_text() for p in soup.find_all('p')])

    stopwords_german = set(stopwords.words('german'))
    text_content = re.sub(r'\W+', ' ', text_content)
    text_content = re.sub(r'\d+', '', text_content)
    tokens = word_tokenize(text_content.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords_german and token not in excluded_words and token not in excluded_months]   # Remove stopwords

    preprocessed_text = " ".join(filtered_tokens)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([preprocessed_text])
    feature_names = vectorizer.get_feature_names_out()

    top_indices = tfidf_matrix.toarray().argsort()[0][-50:]

    top_words = [feature_names[idx] for idx in top_indices]
    df.at[i, 'Top 50 Words'] = ', '.join(top_words)

print(df)

In [ ]:
# WORD CLOUD

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

all_words = df['Top 50 Words'].str.cat(sep=', ')

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([all_words])
feature_names = vectorizer.get_feature_names_out()

top_indices = tfidf_matrix.toarray().argsort()[0][-1000:]

top_words = [feature_names[idx] for idx in top_indices]

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(top_words))

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# DATAFRAME COUR DE CASSATION

NB. Please also refer to the other colab on the Cour de Cassation for other graphs. The dataframes are the same, we just worked on it in different files and used different dataframe names so we could not include the other graphs below.

In [ ]:
### FRANCE ###

from requests_oauthlib import OAuth2Session
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import regex as re
import urllib.parse

API_HOST = "https://sandbox-api.aife.economie.gouv.fr"
TOKEN_URL = 'https://sandbox-oauth.aife.economie.gouv.fr/api/oauth/token'
client_id = "b800ee81-4b08-4b7c-9cb6-67f51535888e"
client_secret = "0adff97c-722d-40aa-bbd6-3e32a08c6ae3"

tokencall = requests.post(TOKEN_URL, data={"grant_type": "client_credentials","client_id": client_id, "client_secret": client_secret, "scope": "openid"})

token = tokencall.json()
client = OAuth2Session(client_id, token=token)


keywords = ["règlement UE", "directive", "CHARTE DES DROITS FONDAMENTAUX DE L'UNION EUROPÉENNE", "TFUE", "Traité sur le fonctionnement de l'Union européenne", "Traité sur l'Union européenne"]

all_df_france = []
for key in keywords:
    base_url = "https://sandbox-api.piste.gouv.fr/cassation/judilibre/v1.0/search?query=" + urllib.parse.quote(
        key) +  "&date_start=2018-01-01&date_end=2023-01-01&page_size=50&page="

    df_france = ["", ""]
    ii = 0
    while len(df_france) > 0 and ii < 150:

        resp = client.get(base_url + str(ii))
        results = resp.json()
        df_france = pd.DataFrame(results["results"])
        df_france['keyword'] = key
        all_df_france.append(df_france)
        ii += 1
df_france = pd.concat(all_df_france)

df_france

In [ ]:
decision_urls = []

for index, row in df_france.iterrows():
    decision_id = row['id']
    decision_url = f"https://www.courdecassation.fr/decision/{decision_id}"
    decision_urls.append(decision_url)

df_france['decision_url'] = decision_urls

df_france['decision_url']

# I. What are the most common french law areas (themes) that reference EU law?

In [ ]:
import pandas as pd
import numpy as np

all_themes = [theme for themes in df_france['themes'] if isinstance(themes, np.ndarray) for theme in themes]

theme_counts = pd.Series(all_themes).value_counts()

# Print the most common themes
print(theme_counts.head(50))

#Exclude certain terms
excluded_terms = ["preuve", "appréciation","délai",'union europeenne', "détermination", "portée", "cas", "applications diverses", "défaut", "critères", "caractérisation", "condition", "effets", "etendue", "application", "contenu", "conditions", "exclusion", "définition", "possibilité", "effet"]

exploded_themes = df_france.explode("themes")

exploded_themes = df_france[["id", "number", "solution", "themes"]].explode("themes")
filtered_themes = exploded_themes[~exploded_themes.isin(excluded_terms)]

filtered_themes = filtered_themes.drop_duplicates()

theme_counts = filtered_themes["themes"].value_counts()
theme_counts.head(50)

In [ ]:
### GRAPH FOR TOP THEMES IN CASS DECISIONS

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
top_themes = theme_counts.head(50)
sns.barplot(x=top_themes.values, y=top_themes.index, palette="viridis")

plt.xlabel('Frequency')
plt.ylabel('Themes')
plt.title('Top 50 Most Common Themes')
plt.show()

We first wanted to find what themes or areas of French national law made the most references to EU law in their decisions. This was done by using the function “explode” to separate the list of themes into individual elements.

When first rendered, the most common terms were often terms that had little actual meaning within an EU context (such as "preuve"), so we decided to exclude certain themes.

We see from the results that an  overall theme is contract and labour law, with many individual themes such as “enterprise en difficulté”, "contrat de travail rupture”, “statut collectif du travail”, “représentation des salariés”.

Especially interesting in the COVID context is the “santé publique” term listed in the top 50 most common terms.

  * Travel within and outside French borders was heavily restricted during the pandemic between March 2020 and June 2021 for reasons of national public health in order to restrict the spread of the virus.
  * However, these restrictions had to stay in compliance with the freedom of movement, one of the four pillars of the EU, which explains why a matter of national public health would be heavily linked with EU law.
  https://www.eca.europa.eu/Lists/ECADocuments/SR22_13/SR_free-movement-phase-I_EN.pdf

#II. Case Outcomes

#Does referencing EU law have an impact on how cases are decided?

In [ ]:
unique_solutions_counts = df_france[["id", "number", "solution"]].drop_duplicates()["solution"].value_counts()
unique_solutions_counts

In [ ]:
#Bar graph
percentage_counts = df_france[["id", "number", "solution"]].drop_duplicates()["solution"].value_counts(normalize=True) * 100

fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(
    data=percentage_counts.reset_index(),
    y="index",
    x="solution",
    palette="magma",
    ax=ax
)

ax.set(xlabel="Percentage (%)", ylabel="Case Outcome")
plt.title('Number of cases per case outcome ')
plt.show()

In [ ]:
import datetime

df_france_solutions = df_france[["id", "number", "solution", "decision_date"]].drop_duplicates() # duplicates made from keyword column
df_france_solutions["solution"] = df_france_solutions["solution"].replace({'qpcother': 'qpc', "rabat": "other", "decheance": "other", "nonlieu": "other", "renvoi": "other"})
print(df_france_solutions)

solution_counts = df_france_solutions['solution'].value_counts()
solution_counts

In [ ]:
#Bar graph
percentage_counts = df_france_solutions['solution'].str.lower().value_counts(normalize=True) * 100

fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(
    data=percentage_counts.reset_index(),
    y="index",
    x="solution",
    palette="magma",
    ax=ax
)

ax.set(xlabel="Percentage (%)", ylabel="Case Outcome")
plt.title('Number of cases per case outcome ')
plt.show()

We first listed the different types of outcomes for cases, which rendered 13 different types. We decided to limit this to the main ones allowing for easier visualisation of the results and grouped the outcomes with fewer results into the already existing category “other”. So we focused on: rejet, cassation, other, npc, irrecevabilité, annulation, avis and désignation.

This clearly showed that the largest proportion of cases are dismissed, at 64%, followed by cassation at just over 30%. It is also interesting to note that the fourth most common outcome is “qpc” (although technically third largest because “other” is made up of multiple possible outcomes), highlighting the role of the Cour de Cassation in seizing the Conseil constitutionnel on a matter of constitutionality (although the number of QPC decisions remains quite low).

This is also  interesting in an EU setting, as the question of EU supra-constitutionality is a growing matter of debate for national courts and jurisdictions.

#How are cases decided over time?


In [ ]:
df_france_solutions["decision_date"] = pd.to_datetime(df_france_solutions["decision_date"])

# Grouping by 'decision_date' and 'solution' and counting number of cases per month
df_france_sol_group = df_france_solutions.set_index("decision_date")[["solution", "id", "number"]].groupby([pd.Grouper(freq='1M'), 'solution']).count().reset_index()
df_france_sol_group["decision_month"] = df_france_sol_group["decision_date"].dt.month

plt.figure(figsize=(15, 8))

ax = sns.lineplot(
    x="decision_date",
    y="number",
    hue="solution",
    palette=sns.color_palette("magma", n_colors=5),
    data=df_france_sol_group,
)

# Line indicating the start of Covid
plt.axvline(x=datetime.datetime(2020, 3, 1), color="green", label='COVID-19')

ax.set(xlabel="Decision date", ylabel="Number of cases")
plt.legend(loc="upper left")
plt.show()

We then plotted the filtered solutions against decision dates, to see if there was a change in outcomes over time. Instead of showing outcomes per specific date we grouped the dates per month, allowing for clearer visualisation.

The graph clearly shows how the majority of cases are dismissed. There was a massive spike in dismissals in early 2019 (it is unclear why exactly, although the Courts adopted new policies in 2019 about the physical formatting/rendering of  decisions, and people may have been able to better understand how to 'seize' the court).

We do clearly notice the effect of COVID 19 on the Court and the rendering of decisions, with a drastic drop in activity in early 2020. This did not however affect the outcomes of the decisions themselves (proportionally), except for a spike in dismissals in late 2020, perhaps as a result of earlier inactivity and because of claims of individual rights' violations under restrictive COVID measures.

#Is there a relationship between the different areas of French law that reference  EU law and the case outcome?

In [ ]:
df_france_exploded = df_france[["id", "number", "solution", "themes"]].explode("themes")
df_france_exploded = df_france_exploded[~df_france_exploded.isin(excluded_terms)]

df_france_exploded["solution"] = df_france_exploded["solution"].replace({'qpcother': 'qpc', "rabat": "other", "decheance": "other", "nonlieu": "other", "renvoi": "other"})

df_france_exploded = df_france_exploded.drop_duplicates() # drop duplicates after because

df_france_themes = df_france_exploded.groupby(["themes", "solution"]).size().to_frame(name = "counts").reset_index().sort_values("counts", ascending=False)
df_france_themes["prop"] = df_france_themes["counts"] / df_france_themes.groupby("themes")["counts"].transform("sum")
df_france_themes = df_france_themes.merge(theme_counts.to_frame(name = "unique_counts").rename_axis("themes").reset_index())

df_france_themes

Important to note here is that the unique themes are cross-referenced with unique solutions. More individual themes lead to cassations, which does not however mean there are more overall cassations (indeed, there are more dismissals).

In [ ]:
df_france_exploded.groupby(["themes", "solution"]).size().to_frame(name = "counts").reset_index().sort_values("counts", ascending=False)

The new data frame here consists of the filtered and unique themes and filtered case outcomes. Each theme was then cross-referenced with the different outcomes, allowing us to count how many times a specific theme led to a specific solution. For instance, “domaine d’application”  led to 179 cassations, 162 dismissals, etc.

In [ ]:
df_themes_plot = df_france_themes[df_france_themes["unique_counts"] >= 100]
df_themes_plot["Case outcome"] = df_themes_plot["solution"]
df_themes_plot["perc"] = df_themes_plot["prop"] * 100

plt.figure(figsize=(15,8))

ax = sns.histplot(
    df_themes_plot,
    y='themes',
    weights="perc",
    hue="Case outcome",
    multiple='stack',
    palette=sns.color_palette("magma", n_colors=7),
    edgecolor='white',
    shrink=0.8
)

ax.set(xlabel="Percentage of total cases (%)", ylabel="Theme of law")

plt.show()

In [ ]:
df_themes_plot = df_france_themes[df_france_themes["unique_counts"] >= 50]
df_themes_plot["Case outcome"] = df_themes_plot["solution"]

plt.figure(figsize=(15,15))

ax = sns.histplot(
    df_themes_plot,
    y='themes',
    weights="counts",
    hue="Case outcome",
    multiple='stack',
    palette=sns.color_palette("magma", n_colors=7),
    edgecolor='white',
    shrink=0.8
)

ax.set(xlabel="Total cases", ylabel="Theme of law")

plt.show()

The data was rendered into a stacked bar graph, both for percentage value and total number of cases. What we see here that the majority of cases lead to cassation, in contrast to the overall higher number of rejections by the court. This is because the data was filtered by the count of unique themes, which have a different distribution of outcomes.

Interesting here is the relatively high percentage of cases referring to EU law that led to a QPC in the area of "licenciement". This could be because EU law has better protection for unfair dismissals (Under Charter of Fundamental rights: "la protection en cas de licenciement injustifié (article 30)). This also points to th conflicts of jurisdiction between national courts and 'supra-constitutional' entities like the ECJ, which national courts will often resist. Indeed, the CC held that EU directives must be adopted within French law but that it could not go against the ‘consitutional identity’ of France.
(https://fra.europa.eu/sites/default/files/fra_uploads/fra-2019-eu-charter-in-france_fr.pdf)  

#III. References to EU law

##Has there been a change in the EU instrument used in the decisions of the Cour de Cassation?

##Is there a relationship between the reference to specific EU instruments in decisions and the outcome of the decision over time?
  
##Is there a link with COVID?

In [ ]:
df_france["keyword"].unique()

In [ ]:
df_france['keyword'] = df_france['keyword'].replace({"TFUE": "Traité sur le fonctionnement de l'Union européenne"})

keyword_counts = df_france[["id", "number", "solution", "keyword"]].drop_duplicates()['keyword'].value_counts()

keyword_counts

In [ ]:
df_france["decision_date"] = pd.to_datetime(df_france["decision_date"])

In [ ]:
df_france_line = df_france[["id", "number", "solution", "decision_date", "keyword"]].drop_duplicates()
df_france_line = df_france_line.set_index("decision_date").groupby([pd.Grouper(freq='1M'), 'keyword']).count().reset_index()
df_france_line["decision_month"] = df_france_line["decision_date"].dt.month
df_france_line

In [ ]:
import datetime
plt.figure(figsize=(15,8))

ax = sns.lineplot(
    x="decision_date",
    y="number",
    hue="keyword",
    palette=sns.color_palette("viridis", n_colors=5),
    data=df_france_line,
)

# line at Covid
plt.axvline(x=datetime.datetime(2020, 3, 1), color="red", label="COVID 19")

ax.set(xlabel="Decision date", ylabel="Number of cases")
plt.legend(loc="upper left")
plt.show()

The Traité du fonctionnement de l'UE is most referenced in Cour de Cassation decisions.

COVID shows a drastic drop in decisions rendered, but also shows that immediately following the crisis there was greater reference to the Charter of Fundamental Rights of the EU proportionally to other years. This can be explained by the number of cases that were brought forth claiming breaches of rights caused by the strict protective measures of the state during this time,  especially on the freedom of movement, which is a fundamental right of the Charter (https://www.frontiersin.org/articles/10.3389/fpos.2020.618664/full).

The overall number of cases is much lower than on the graph that plots outcomes over time because we only filtered for a number of EU instruments and not all. This could of course be amended when looking for a specific EU instrument.

In [ ]:
# outcomes and EU instrument
df_france_solutions = df_france[["id", "number", "solution", "decision_date", "keyword"]].drop_duplicates()
df_france_solutions["solution"] = df_france_solutions["solution"].replace({'qpcother': 'qpc', "rabat": "other", "decheance": "other", "nonlieu": "other", "renvoi": "other"})
df_france_solutions["decision_date"] = pd.to_datetime(df_france_solutions["decision_date"])

df_france_grouped = df_france_solutions.groupby([pd.Grouper(key='decision_date', freq='1M'), 'keyword', 'solution']).count().reset_index()

plt.figure(figsize=(15, 8))

ax = sns.lineplot(
    x="decision_date",
    y="id",
    hue="solution",
    style="keyword",
    markers=True,
    dashes=False,
    palette=sns.color_palette("rocket", n_colors=len(df_france_grouped['keyword'].unique())),
    data=df_france_grouped
)

plt.axvline(x=datetime.datetime(2020, 3, 1), color="green", label='COVID-19')

ax.set(xlabel="Decision date", ylabel="Number of cases")
plt.legend(title='Solution', loc='upper left')
plt.show()

The keyword column has duplicates, which means each case does not have a unique keyword. So we are plotting articles coloured by keyword, but the same article can have multiple keywords.

The general trend seems to be that no matter what EU instrument is referred to in the decision, it will lead to a dismissal (although there is more variation recently).

The important exception to this trend is that the Charter of Fundamental Rights was the most cited EU document when decisions led to a cassation in late 2020.

This highlights again the impact of COVID on decisions of the Cour de cassation and especially within the EU context. Following COVID and the restrictions many governments imposed, there were many claims of violations of individual freedoms, which are enshrined in the Charter.

# DATAFRAME ECJ

In [ ]:
### ECJ ###

import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

urls = [
    'https://curia.europa.eu/juris/documents.jsf?page=1&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=357337',
    'https://curia.europa.eu/juris/documents.jsf?page=2&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=365912',
    'https://curia.europa.eu/juris/documents.jsf?page=3&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=365912',
    'https://curia.europa.eu/juris/documents.jsf?page=4&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=365912',
    'https://curia.europa.eu/juris/documents.jsf?page=5&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=365912',
    'https://curia.europa.eu/juris/documents.jsf?page=6&nat=or&mat=or&pcs=Oor&jur=C&for=&jge=&dates=%2524type%253Dpro%2524mode%253DfromTo%2524from%253D2018.01.01%2524to%253D2023.01.01&language=en&pro=&etat=clot&cit=none%252CC%252CCJ%252CR%252C2008E%252C%252C%252C%252C%252C%252C%252C%252C%252C%252Ctrue%252Cfalse%252Cfalse&oqp=&td=%3BALL&avg=&lgrec=en&parties=allemagne%252C%2Bfrance&lg=&cid=365912'
]

dfs = []

for url in urls:
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text

        soup = BeautifulSoup(html_content, 'html.parser')
        table = soup.find('table', class_='detail_table_documents')

        if table:
            column_aff_data = table.find_all('td', class_='table_cell_aff')
            column_aff_values = [data.get_text(strip=True) for data in column_aff_data]

            df_Case = pd.DataFrame({'Case': column_aff_values})
            df_Case = df_Case[df_Case['Case'] != '']
            df_Case.reset_index(drop=True, inplace=True)

            dfs.append(df_Case)

df_Case = pd.concat(dfs, ignore_index=True)

text_segments_per_cell = []
all_dates = []
all_data = []

pattern = r'(Request|Application|Order|Judgment|Opinion|Removal|Abstract)'

for url in urls:
    response = requests.get(url)

    if response.status_code == 200:
        html_content = response.text

        soup = BeautifulSoup(html_content, 'html.parser')
        table = soup.find('table', class_='detail_table_documents')


        date_elements = soup.find_all(class_='table_cell_date')
        dates = [date.get_text(strip=True) for date in date_elements]
        all_dates.extend(dates[1:])


        elements = soup.find_all(class_='table_cell_links_curia')
        data = [element.get_text(strip=True) for element in elements]
        all_data.extend(data[1:])

        if table:
            rows = table.find_all('tr')

            for row in rows:
                cells = row.find_all('td', class_='table_cell_doc')

                if any(cells):
                    row_text_segments = []

                    for cell in cells:
                        cell_text = cell.get_text(strip=True)


                        matches = re.findall(pattern, cell_text)

                        row_text_segments.append(''.join(matches))

                    if any(row_text_segments):
                        text_segments_per_cell.append(row_text_segments[0])

if not text_segments_per_cell:
    print("No text segments found.")
else:
    df_ECJ_final = pd.DataFrame({"Dates": all_dates, "Case Outcome": text_segments_per_cell, "Subject": all_data, "Case Number": df_Case["Case"].values})

df_ECJ_final

In [ ]:
#ECJ judgments

judgment = df_ECJ_final.apply(lambda row: 'Judgment' in ''.join(row), axis=1)
df_ECJ_judgment = df_ECJ_final[judgment].reset_index(drop=True)

In [ ]:
#Filtered judgments to only have 1 per case number

df_ECJ_judgment_no_duplicates = df_ECJ_judgment.drop_duplicates(subset='Case Number')
df_ECJ_judgment_filtered = df_ECJ_judgment_no_duplicates.reset_index(drop=True)
df_ECJ_judgment_filtered

In [ ]:
from collections import Counter

df = []

#Split themes
unique_subjects = df_ECJ_judgment_filtered['Subject'].unique().tolist()
str(unique_subjects)
split_subjects = [subject.split('-') for subject in unique_subjects]
flattened_list = [item for sublist in split_subjects for item in sublist]
unique_split_themes = list(set(flattened_list))
unique_split_themes
element_counts = Counter(unique_split_themes)
element_counts